In [ ]:
#import requests
#from bs4 import BeautifulSoup

In [ ]:
# #load composite material bridges
# url = "https://structurae.net/en/structures/bridges/composite-bridges/list"
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'html.parser')
# table_composite = soup.find('table', {id: 'structureListResults'})

# #headers = [header.text.strip() for header in table_composite.find_all('th')]

# print(response.text)

# # rows = []
# # for row in table.find_all('tr'):
# #     cells = [cell.text.strip() for cell in row.find_all('td')]
# #     if cells:
# #         rows.append(cells)
# # print(headers)
# # for row in rows:
# #     print(row)

In [ ]:
#!pip install selenium
#!apt-get update 
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin/

In [1]:
%%shell
# DISCONNECT AND DELETE RUNTIME BEFORE RUNNING
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install --reinstall chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.xcONJ67IDi/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.65a4x6Ua3K/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.37dk1q5Bdv/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [2]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import pandas as pd

In [ ]:
url = 'https://structurae.net/en/structures/bridges/composite-bridges/list'

#service = Service("/usr/bin/chromedriver")
#service = Service("/usr/lib/chromium-browser/chromedriver")

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-dev-shm-usage')

#driver = webdriver.Chrome(service=service, options=options)
driver = webdriver.Chrome(service = Service("/usr/bin/chromedriver"), options = options)

# Load the website
driver.get(url)

# Wait for the table to load (optional)
# You may need to adjust the timeout depending on your internet speed and computer performance
driver.implicitly_wait(10)

# Get number of pages for table
num_pages = int(driver.find_element(By.CLASS_NAME, 'pageLinksPageNav').find_elements(By.TAG_NAME, 'a')[-1].get_attribute("data-page"))
print(num_pages)

# Extract the headers
table = driver.find_element(By.CLASS_NAME, 'mobile-scroll-table')
headers = [header.text.strip() for header in table.find_elements(By.TAG_NAME, 'th')]

headers.append('Type')

print(headers)

#driver.quit()


rows = []

for i in range(0, num_pages * 100, 100):
    url = 'https://structurae.net/en/structures/bridges/composite-bridges/list?min={}'.format(i)
    #driver = webdriver.Chrome(service = Service("/usr/bin/chromedriver"), options = options)
    driver.get(url)
    driver.implicitly_wait(10)

    # Find the table element
    table = driver.find_element(By.CLASS_NAME, 'mobile-scroll-table')

    # Extract the rows
    for row in table.find_elements(By.TAG_NAME, 'tr'):
        i = 0
        row_data = []
        for cell in row.find_elements(By.TAG_NAME, 'td'):
            if(i == 4):
                img = cell.find_element(By.TAG_NAME, 'img')
                item = img.get_attribute("alt")#.text.strip()
                row_data.append(item)
            else:
                item = cell.text.strip()
                row_data.append(item)
            i += 1
        #cells = [cell.text.strip() for cell in row.find_elements(By.TAG_NAME, 'td')]
        if row_data:
            row_data.append('composite')
            rows.append(row_data)

    # Close the browser
    #driver.quit()

# Print the headers and rows
for row in rows:
    print(row)

#Create csv
df = pd.DataFrame(rows, columns = headers)
df.to_csv('/usr/bin/bridges.csv', index = False, encoding = 'utf-8-sig')

16
['#', 'Name', 'Year', 'Location', 'Country', 'Status', 'Type']
['1', '470 Odra River Bridge', '2009', 'Ostrava', 'Czechia', 'in use', 'composite']
['2', '4708/C203 Bridge', '2009', 'Ostrava', 'Czechia', 'in use', 'composite']
['3', '7th Avenue Pedestrian Bridge (N1)', '2010', 'Johannesburg', 'South Africa', 'in use', 'composite']
['4', 'A 1 Lenne Bridge', '2009', 'Hagen (NRW)', 'Germany', 'in use', 'composite']
['5', 'A 113 Teltowkanal Bridge', '2004', '- Berlin (BE)', 'Germany', 'in use', 'composite']
['6', 'A 30 Werre East Bridge', '2012', 'Bad Oeynhausen (NRW)', 'Germany', 'in use', 'composite']
['7', 'A 30 Werre West Bridge', '2012', 'Löhne (NRW)', 'Germany', 'in use', 'composite']
['8', 'A 31 Lippe River Bridge', '1981', 'Dorsten (NRW)', 'Germany', 'in use', 'composite']
['9', 'A-8 Directional Ramp Bridge', '', 'Bilbao', 'Spain', 'in use', 'composite']
['10', 'A-8 Exit 115A Ramp Bridge', '', 'Bilbao', 'Spain', 'in use', 'composite']
['11', 'A1 Area Tramway Bridge', '', 'Stuttga

In [ ]:
url = 'https://structurae.net/en/structures/bridges/composite-material-bridges/list'

#service = Service("/usr/bin/chromedriver")
#service = Service("/usr/lib/chromium-browser/chromedriver")

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-dev-shm-usage')

#driver = webdriver.Chrome(service=service, options=options)
#driver = webdriver.Chrome(service = Service("/usr/bin/chromedriver"), options = options)

# Load the website
driver.get(url)

# Wait for the table to load (optional)
# You may need to adjust the timeout depending on your internet speed and computer performance
driver.implicitly_wait(10)

# Get number of pages for table
num_pages = int(driver.find_element(By.CLASS_NAME, 'pageLinksPageNav').find_elements(By.TAG_NAME, 'a')[-1].get_attribute("data-page"))
print(num_pages)

# Extract the headers
table = driver.find_element(By.CLASS_NAME, 'mobile-scroll-table')
headers = [header.text.strip() for header in table.find_elements(By.TAG_NAME, 'th')]

headers.append('Type')

print(headers)

#driver.quit()

rows = []

for i in range(0, num_pages * 100, 100):
    url = 'https://structurae.net/en/structures/bridges/composite-material-bridges/list?min={}'.format(i)
    #driver = webdriver.Chrome(service = Service("/usr/bin/chromedriver"), options = options)
    driver.get(url)
    driver.implicitly_wait(10)

    # Find the table element
    table = driver.find_element(By.CLASS_NAME, 'mobile-scroll-table')

    # Extract the rows
    for row in table.find_elements(By.TAG_NAME, 'tr'):
        i = 0
        row_data = []
        for cell in row.find_elements(By.TAG_NAME, 'td'):
            if(i == 4):
                img = cell.find_element(By.TAG_NAME, 'img')
                item = img.get_attribute("alt")#.text.strip()
                row_data.append(item)
            else:
                item = cell.text.strip()
                row_data.append(item)
            i += 1
        #cells = [cell.text.strip() for cell in row.find_elements(By.TAG_NAME, 'td')]
        if row_data:
            row_data.append('composite')
            rows.append(row_data)

    # Close the browser
    #driver.quit()

# Print the headers and rows
for row in rows:
    print(row)
#Append data to csv
df2 = pd.DataFrame(rows, columns = headers)
df2.to_csv('/usr/bin/bridges.csv', mode='a', index = False, header = False, encoding = 'utf-8-sig')

1
['#', 'Name', 'Year', 'Location', 'Country', 'Status', 'Type']
['1', 'Aberfeldy Footbridge', '1992', 'Aberfeldy (GB-SCT)', 'United Kingdom', 'in use', 'composite']
['2', 'Ait Bayoud Footbridge', '2013', 'Morocco', 'Morocco', 'in use', 'composite']
['3', 'Biobridge', '2016', 'Eindhoven', 'Netherlands', 'in use', 'composite']
['4', 'Bradkirk Footbridge', '2009', 'Blackpool (GB-ENG)', 'United Kingdom', 'in use', 'composite']
['5', 'Chamberlain Bridge', '2006', 'Bridgetown', 'Barbados', 'in use', 'composite']
['6', 'Fiberline Bridge', '1997', 'Kolding', 'Denmark', 'in use', 'composite']
['7', 'Footbridge over the Kempt', '2001', 'Winterthur (ZH)', 'Switzerland', 'in use', 'composite']
['8', 'Forrest & Maxie Preston Memorial Suspension Bridge', '1999', 'River (KY)', 'USA', 'in use', 'composite']
['9', 'Foryd Harbour Footbridge', '2013', 'Rhyl (GB-WLS)', 'United Kingdom', 'in use', 'composite']
['10', 'Fredrikstad Footbridge', '2006', 'Fredrikstad', 'Norway', 'in use', 'composite']
['11', 

In [ ]:
url = 'https://structurae.net/en/structures/bridges/concrete-bridges/list'

#service = Service("/usr/bin/chromedriver")
#service = Service("/usr/lib/chromium-browser/chromedriver")

options = Options()
options.add_argument("--headless")
#options.headless = True
options.add_argument("--no-sandbox")
options.add_argument('--disable-dev-shm-usage')

#driver = webdriver.Chrome(service=service, options=options)
#driver = webdriver.Chrome(service = Service("/usr/bin/chromedriver"), options = options)

# Load the website
driver.get(url)

# Wait for the table to load (optional)
# You may need to adjust the timeout depending on your internet speed and computer performance
driver.implicitly_wait(10)

# Get number of pages for table
num_pages = int(driver.find_element(By.CLASS_NAME, 'pageLinksPageNav').find_elements(By.TAG_NAME, 'a')[-1].get_attribute("data-page"))
print(num_pages)

# Extract the headers
table = driver.find_element(By.CLASS_NAME, 'mobile-scroll-table')
headers = [header.text.strip() for header in table.find_elements(By.TAG_NAME, 'th')]

headers.append('Type')

print(headers)

rows = []

for i in range(0, num_pages * 100, 100):
    url = 'https://structurae.net/en/structures/bridges/concrete-bridges/list?min={}'.format(i)
    #driver = webdriver.Chrome(service = Service("/usr/bin/chromedriver"), options = options)
    driver.get(url)
    driver.implicitly_wait(10)

    # Find the table element
    table = driver.find_element(By.CLASS_NAME, 'mobile-scroll-table')

    # Extract the rows
    for row in table.find_elements(By.TAG_NAME, 'tr'):
        i = 0
        row_data = []
        for cell in row.find_elements(By.TAG_NAME, 'td'):
            if(i == 4):
                img = cell.find_element(By.TAG_NAME, 'img')
                item = img.get_attribute("alt")#.text.strip()
                row_data.append(item)
            else:
                item = cell.text.strip()
                row_data.append(item)
            i += 1
        #cells = [cell.text.strip() for cell in row.find_elements(By.TAG_NAME, 'td')]
        if row_data:
            row_data.append('concrete')
            rows.append(row_data)

    # Close the browser
    #driver.quit()

#driver.quit()

# Print the headers and rows
for row in rows:
    print(row)

#Append data to csv
df2 = pd.DataFrame(rows, columns = headers)
df2.to_csv('/usr/bin/bridges.csv', mode='a', index = False, header = False, encoding = 'utf-8-sig')

Streaming output truncated to the last 5000 lines.
['995', 'Cleft-Ridge Span', '1872', 'New York (NY)', 'USA', 'in use', 'concrete']
['996', 'Clemmer Bridge', '', 'Perth East (ON)', 'Canada', 'in use', 'concrete']
['997', 'Clichy Bridge', '1975', 'Clichy (92)', 'France', 'in use', 'concrete']
['998', 'Clichy Metro Viaduct', '1980', 'Clichy (92)', 'France', 'in use', 'concrete']
['999', 'Clidane Viaduct', '1999', 'Bourg-Lastic (63)', 'France', 'in use', 'concrete']
['1000', 'Clifton Avenue Bridge', '2002', 'Baltimore (MD)', 'USA', 'in use', 'concrete']
['1001', 'Clifton Bridge', '1967', 'Penrith (GB-ENG)', 'United Kingdom', 'in use', 'concrete']
['1002', 'Clifton Bridge', '1958', 'Nottingham (GB-ENG)', 'United Kingdom', 'in use', 'concrete']
['1003', 'Clifton Bridge', '1972', 'Nottingham (GB-ENG)', 'United Kingdom', 'in use', 'concrete']
['1004', 'Climax Bridge', '', 'Climax (KS)', 'USA', 'demolished', 'concrete']
['1005', 'Cline Avenue Bridge', '2020', 'East Chicago (IN)', 'USA', 'in u

In [4]:
url = 'https://structurae.net/en/structures/bridges/metal-bridges/list'

#service = Service("/usr/bin/chromedriver")
#service = Service("/usr/lib/chromium-browser/chromedriver")

options = Options()
options.add_argument("--headless")
#options.headless = True
options.add_argument("--no-sandbox")
options.add_argument('--disable-dev-shm-usage')

#driver = webdriver.Chrome(service=service, options=options)
driver = webdriver.Chrome(service = Service("/usr/bin/chromedriver"), options = options)

# Load the website
driver.get(url)

# Wait for the table to load (optional)
# You may need to adjust the timeout depending on your internet speed and computer performance
driver.implicitly_wait(10)

# Get number of pages for table
num_pages = int(driver.find_element(By.CLASS_NAME, 'pageLinksPageNav').find_elements(By.TAG_NAME, 'a')[-1].get_attribute("data-page"))
print(num_pages)

# Extract the headers
table = driver.find_element(By.CLASS_NAME, 'mobile-scroll-table')
headers = [header.text.strip() for header in table.find_elements(By.TAG_NAME, 'th')]

headers.append('Type')

print(headers)

rows = []

for i in range(0, num_pages * 100, 100):
    url = 'https://structurae.net/en/structures/bridges/metal-bridges/list?min={}'.format(i)
    #driver = webdriver.Chrome(service = Service("/usr/bin/chromedriver"), options = options)
    driver.get(url)
    driver.implicitly_wait(10)

    # Find the table element
    table = driver.find_element(By.CLASS_NAME, 'mobile-scroll-table')

    # Extract the rows
    for row in table.find_elements(By.TAG_NAME, 'tr'):
        i = 0
        row_data = []
        for cell in row.find_elements(By.TAG_NAME, 'td'):
            if(i == 4):
                img = cell.find_element(By.TAG_NAME, 'img')
                item = img.get_attribute("alt")#.text.strip()
                row_data.append(item)
            else:
                item = cell.text.strip()
                row_data.append(item)
            i += 1
        
        #cells = [cell.text.strip() for cell in row.find_elements(By.TAG_NAME, 'td')]
        if row_data:
            row_data.append('metal')
            rows.append(row_data)

    # Close the browser
    #driver.quit()

driver.quit()

# Print the headers and rows
for row in rows:
    print(row)

#Append data to csv
df2 = pd.DataFrame(rows, columns = headers)
df2.to_csv('/usr/bin/bridges.csv', mode='a', index = False, header = False, encoding = 'utf-8-sig')

Streaming output truncated to the last 5000 lines.
['2958', 'IDEO School Footbridge', '2018', 'Madrid', 'Spain', 'in use', 'metal']
['2959', 'Ignacy Moscicki Bridge', '1934', 'Pulawy (LU)', 'Poland', 'in use', 'metal']
['2960', 'Ignatz Bubis Bridge', '1878', 'Frankfurt (HE)', 'Germany', 'in use', 'metal']
['2961', 'Igollo de Camargo Footbridge', '2010', 'Camargo', 'Spain', 'in use', 'metal']
['2962', 'Ihmebrücke Ricklingen', '', 'Hanover (NI)', 'Germany', 'in use', 'metal']
['2963', 'IJmuiden Swing Bridge', '1956', 'Velsen', 'Netherlands', 'in use', 'metal']
['2964', 'Ijsselkanaal Footbridge', '1964', 'Zwolle', 'Netherlands', 'in use', 'metal']
['2965', 'IJsselkanaal Railroad Bridge', '1964', 'Zwolle', 'Netherlands', 'in use', 'metal']
['2966', 'IJzerlaan Rail Bridge', '2003', 'Antwerp', 'Belgium', 'in use', 'metal']
['2967', 'IJzerlaan Rail Bridges', '2013', 'Antwerp', 'Belgium', 'in use', 'metal']
['2968', 'IKEA Bridge', '', 'Dortmund (NRW)', 'Germany', 'in use', 'metal']
['2969', 'I